In [1]:
from langchain_core.output_parsers import JsonOutputParser
from sympy.concrete import products

from llms import model, ollama_model
from prompts import query_refinement, image_query_extraction, products_choice
from langchain_core.prompts import ChatPromptTemplate

C:\Users\GIGABYTE\PycharmProjects\VectorsHackthon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def refine(query:str):
    prompt = ChatPromptTemplate.from_template(query_refinement)
    chain = prompt | model | JsonOutputParser()
    try:
        answer = chain.invoke({"query": query})
        print(answer)
        return answer
    except Exception as e:
        return {"error": f"Failed to parse: {str(e)}", "raw_query": query}

In [3]:
res = refine("a leather jacket ")

{'semantic_query': 'leather jacket Apparel leather jacket', 'filters': {'max_price': 800, 'monthly_cash_flow': 0, 'category': 'Apparel'}, 'keywords': ['leather', 'jacket', 'apparel']}


In [4]:
res

{'semantic_query': 'leather jacket Apparel leather jacket',
 'filters': {'max_price': 800, 'monthly_cash_flow': 0, 'category': 'Apparel'},
 'keywords': ['leather', 'jacket', 'apparel']}

In [5]:
import base64
from langchain_core.messages import HumanMessage
def describe_image(image_path: str):
    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")


    message = HumanMessage(
        content=[
            {"type": "text", "text": image_query_extraction},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
            },
        ]
    )

    try:
        response = model.invoke([message])
        return response.content
    except Exception as e:
        return f"Error describing image: {str(e)}"

In [6]:
print(describe_image("camera.jpg"))

KeyboardInterrupt: 

In [6]:
from Hybrid_Search import HybridSearcher
hybrid_searcher = HybridSearcher(collection_name="products")

In [7]:
def workflow(query):
    refined_query = refine(query)
    text = refined_query["semantic_query"]
    print(refined_query)
    result = hybrid_searcher.search(text)
    return result

In [8]:
print (workflow("cheap jacket for men"))

{'semantic_query': "jacket, men's clothing, cheap", 'filters': {'max_price': 70, 'monthly_cash_flow': 0, 'category': "men's jackets"}, 'keywords': ['jacket', 'men', 'cheap', 'affordable', 'budget', 'male']}
{'semantic_query': "jacket, men's clothing, cheap", 'filters': {'max_price': 70, 'monthly_cash_flow': 0, 'category': "men's jackets"}, 'keywords': ['jacket', 'men', 'cheap', 'affordable', 'budget', 'male']}
[{'description': "product : Lint Remover For Clothes With 1 Year Warranty Fabric Shaver Lint Shaver for Woolen Clothes Blanket Jackets Stainless Steel Blades,Bedding, Clothes and Furniture Best Remover for Fabrics Portable Lint Shavers (White Orange)category : Home&Kitchen|Kitchen&HomeAppliances|Vacuum,Cleaning&Ironing|Irons,Steamers&Accessories|LintShaversdetails : Our Lint Remover for Clothes MAKE ALL FABRICS LOOK BRAND NEW- SaiEllin Electric Lint Remover Smoothly takes off the Bur, Lint Remover, Fuzz Cleaner, Bubble Shaver, Lintremover, Fur Fuss Pilling, Bhur, Bubbling Cleanin

In [3]:
refined_query = refine("Cheap gaming keyboard for under 600 dollars ")

{'semantic_query': 'gaming keyboard keyboard cheap', 'filters': {'max_price': 599.99, 'monthly_cash_flow': None, 'category': 'keyboard'}, 'keywords': ['cheap', 'gaming', 'keyboard']}


In [4]:
print(refined_query)

{'semantic_query': 'gaming keyboard keyboard cheap', 'filters': {'max_price': 599.99, 'monthly_cash_flow': None, 'category': 'keyboard'}, 'keywords': ['cheap', 'gaming', 'keyboard']}


In [5]:
from qdrant_client import models
query_filter=models.Filter(
                should=[
                    models.FieldCondition(
                        key="discounted_price",
                        range=models.Range(lte=refined_query["filters"]["max_price"]),
                    )
                ],
            )

In [6]:
from qdrant_client import QdrantClient, models


class HybridSearcher:
    DENSE_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    SPARSE_MODEL = "Qdrant/bm25"
    LATE_INTERACTION_MODEL = "colbert-ir/colbertv2.0"

    def __init__(self, collection_name):
        self.collection_name = collection_name
        self.qdrant_client = QdrantClient(url="http://localhost:6333")


    def search(self, text,filters):
        search_result = self.qdrant_client.query_points(
            collection_name=self.collection_name,
            prefetch=[
                models.Prefetch(
                    query=models.Document(text=text, model=self.DENSE_MODEL),
                    using="text-dense",
                    limit=5
                ),
                models.Prefetch(
                    query=models.Document(text=text, model=self.SPARSE_MODEL),
                    using="text-sparse",
                    limit=5
                ),
            ],
            query=models.Document(text=text, model=self.LATE_INTERACTION_MODEL),
            using="text-late-interaction",
            with_payload=True,
            query_filter=filters,
            limit=5,
        ).points
        metadata = [point.payload for point in search_result]
        return metadata
hybrid_searcher2 = HybridSearcher("products")

In [7]:
result = hybrid_searcher2.search(text=refined_query["semantic_query"],filters=query_filter)

In [8]:
print(result)

[{'description': "product : Redragon K617 Fizz 60% Wired RGB Gaming Keyboard, 61 Keys Compact Mechanical Keyboard w/White and Grey Color Keycaps, Linear Red Switch, Pro Driver/Software Supportedcategory : Computers&Accessories|Accessories&Peripherals|PCGamingPeripherals|GamingKeyboardsdetails : Slim Fresh K617: Redragon's new 60% layout wired keyboard in a refreshing style with a no-brainer deal economic offer. Compact 61 keys with selected keycaps, dedicated for FPS Gamers and efficient working.|Hot-Swappable Red Switches: Most quiet mechanical switch, linear and soft key travel makes every click easy to register. Hot-swappable with other Redragon switches. Made to last with switches rated for 50 million keypresses.|Vibrant RGB: Up to 20 presets backlighting modes are free to choose by the keyboard itself. Brightness and flowing speed is also adjustable on board. Select your own preferred modes for any playing.|Pro Software Customizable: Expand your options using the available softwar

In [9]:
from prompts import query_refinement, image_query_extraction, products_choice
def reason_for_choice(query,product_list):
    prompt = ChatPromptTemplate.from_template(products_choice)
    chain = prompt | model
    try:
        answer = chain.invoke({"query": query, "product_list": product_list})
        print(answer)
        return answer
    except Exception as e:
        return {"error": f"Failed to parse: {str(e)}", "raw_query": query}

In [10]:
res = reason_for_choice(query=refined_query["semantic_query"],product_list=result)

content='```json\n[\n  {\n    "product_name": "Redragon K617 Fizz 60% Wired RGB Gaming Keyboard",\n    "financial_recommendation": "This Redragon K617 Fizz keyboard is a smart purchase for a user seeking a \'gaming keyboard\' that is \'cheap\'. At a discounted price of 2.649, it offers significant value by providing advanced gaming features such as a compact 60% layout, hot-swappable linear red mechanical switches, vibrant RGB lighting, and dedicated software customization, all while remaining highly affordable for a mechanical gaming keyboard."\n  },\n  {\n    "product_name": "HP K500F Backlit Membrane Wired Gaming Keyboard",\n    "financial_recommendation": "For a user specifically looking for a \'gaming keyboard\' that is \'cheap\', the HP K500F presents an excellent value proposition. Priced at a very accessible 1.149, it delivers essential gaming features including mixed color backlighting, a durable metal panel, and 26 anti-ghosting keys, making it a highly budget-friendly choice

In [12]:
print(res.content)

```json
[
  {
    "product_name": "Redragon K617 Fizz 60% Wired RGB Gaming Keyboard",
    "financial_recommendation": "This Redragon K617 Fizz keyboard is a smart purchase for a user seeking a 'gaming keyboard' that is 'cheap'. At a discounted price of 2.649, it offers significant value by providing advanced gaming features such as a compact 60% layout, hot-swappable linear red mechanical switches, vibrant RGB lighting, and dedicated software customization, all while remaining highly affordable for a mechanical gaming keyboard."
  },
  {
    "product_name": "HP K500F Backlit Membrane Wired Gaming Keyboard",
    "financial_recommendation": "For a user specifically looking for a 'gaming keyboard' that is 'cheap', the HP K500F presents an excellent value proposition. Priced at a very accessible 1.149, it delivers essential gaming features including mixed color backlighting, a durable metal panel, and 26 anti-ghosting keys, making it a highly budget-friendly choice that still meets the cor

In [17]:
l=res.content.split('}')
for el in l :
    print(el)
    print("_______________________________________")

```json
[
  {
    "product_name": "Redragon K617 Fizz 60% Wired RGB Gaming Keyboard",
    "financial_recommendation": "This Redragon K617 Fizz keyboard is a smart purchase for a user seeking a 'gaming keyboard' that is 'cheap'. At a discounted price of 2.649, it offers significant value by providing advanced gaming features such as a compact 60% layout, hot-swappable linear red mechanical switches, vibrant RGB lighting, and dedicated software customization, all while remaining highly affordable for a mechanical gaming keyboard."
  
_______________________________________
,
  {
    "product_name": "HP K500F Backlit Membrane Wired Gaming Keyboard",
    "financial_recommendation": "For a user specifically looking for a 'gaming keyboard' that is 'cheap', the HP K500F presents an excellent value proposition. Priced at a very accessible 1.149, it delivers essential gaming features including mixed color backlighting, a durable metal panel, and 26 anti-ghosting keys, making it a highly budget-